<a href="https://colab.research.google.com/github/SamiraHabib/Coursera_Capstone/blob/main/Coursera_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coursera Capstone #

In [ ]:
import pandas as pd 
import numpy as np 


In [ ]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Import necessary Libraries #

In [61]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


# Define Foursquare Credentials and Version #

In [62]:
CLIENT_ID = '1LFBJCCEY1TSE4R1J4Y2MX5PZMPVS4O4IFXTHHCQQMYT53SA' # your Foursquare ID
CLIENT_SECRET = 'ZL54S51I5OLJWENEGTRHCTKLYQ21GIRDXWCLZUMYS24J4J2D' # your Foursquare Secret
ACCESS_TOKEN = '' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1LFBJCCEY1TSE4R1J4Y2MX5PZMPVS4O4IFXTHHCQQMYT53SA
CLIENT_SECRET:ZL54S51I5OLJWENEGTRHCTKLYQ21GIRDXWCLZUMYS24J4J2D


# Endereço do local escolhido #

In [153]:
#Exemplo - alterar
address = 'Rio de Janeiro'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-22.9110137 -43.2093727


# Explore a location #

In [154]:
#Localidade exemplo, mudar
latitude = -22.9110137
longitude = -43.2093727

In [155]:
search_query = 'High School'
radius = 200.000
print(search_query + ' .... OK!')

High School .... OK!


# Define URL #

In [156]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url
#definir com:
#https://api.foursquare.com/v2/venues/explore?client_id=CLIENT_ID&client_secret=CLIENT_SECRET&ll=LATITUDE,LONGITUDE&v=VERSION&limit=LIMIT

'https://api.foursquare.com/v2/venues/search?client_id=1LFBJCCEY1TSE4R1J4Y2MX5PZMPVS4O4IFXTHHCQQMYT53SA&client_secret=ZL54S51I5OLJWENEGTRHCTKLYQ21GIRDXWCLZUMYS24J4J2D&ll=-22.9110137,-43.2093727&v=20180604&radius=200.0&limit=50'

# Send GET request and examine results #

In [157]:
import requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6081dbb251a32a772ce4ce52'},
 'response': {'confident': False,
  'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/bridge_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d1df941735',
      'name': 'Bridge',
      'pluralName': 'Bridges',
      'primary': True,
      'shortName': 'Bridge'}],
    'hasPerk': False,
    'id': '50c707f7e4b03c61133b0573',
    'location': {'cc': 'BR',
     'city': 'Rio de Janeiro',
     'country': 'Brasil',
     'distance': 77,
     'formattedAddress': ['Rio de Janeiro, RJ', 'Brasil'],
     'labeledLatLngs': [{'label': 'display',
       'lat': -22.910322867434417,
       'lng': -43.20949946081052}],
     'lat': -22.910322867434417,
     'lng': -43.20949946081052,
     'state': 'RJ'},
    'name': 'Viaduto dos Fuzileiros',
    'referralId': 'v-1619123122'},
   {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/bridge_',
       

## Get relevant part of JSON and transform it into a pandas dataframe ##

In [158]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.address,location.postalCode,location.crossStreet,venuePage.id,location.neighborhood
0,50c707f7e4b03c61133b0573,Viaduto dos Fuzileiros,"[{'id': '4bf58dd8d48988d1df941735', 'name': 'B...",v-1619123122,False,-22.910323,-43.209499,"[{'label': 'display', 'lat': -22.9103228674344...",77,BR,Rio de Janeiro,RJ,Brasil,"[Rio de Janeiro, RJ, Brasil]",NaN,NaN,NaN,NaN,NaN
1,50ca435ee4b047cd3cd92e78,Viaduto dos Marinheiros,"[{'id': '4bf58dd8d48988d1df941735', 'name': 'B...",v-1619123122,False,-22.910619,-43.213063,"[{'label': 'display', 'lat': -22.9106185055282...",380,BR,NaN,NaN,Brasil,[Brasil],NaN,NaN,NaN,NaN,NaN
2,5128e368e4b02e93a4821aea,Botafogo Mercado De Flores,"[{'id': '4bf58dd8d48988d11b951735', 'name': 'F...",v-1619123122,False,-22.910233,-43.209925,"[{'label': 'display', 'lat': -22.9102330572094...",103,BR,Rio de Janeiro,RJ,Brasil,"[Rio de Janeiro, RJ, Brasil]",NaN,NaN,NaN,NaN,NaN
3,4d9a43728163a35d8077577e,Centro de Convenções SulAmérica,"[{'id': '4bf58dd8d48988d1ff931735', 'name': 'C...",v-1619123122,False,-22.912070,-43.208819,"[{'label': 'display', 'lat': -22.9120699311280...",130,BR,Rio de Janeiro,RJ,Brasil,"[Av. Paulo de Frontin, 1, Rio de Janeiro, RJ, ...","Av. Paulo de Frontin, 1",20260-010,NaN,NaN,NaN
4,5bc8c85b2b9844002cf18446,Expo Cake Brasil,"[{'id': '4eb1daf44b900d56c88a4600', 'name': 'F...",v-1619123122,False,-22.912203,-43.209106,"[{'label': 'display', 'lat': -22.912203, 'lng'...",135,BR,Rio de Janeiro,RJ,Brasil,"[Rio de Janeiro, RJ, 20210-005, Brasil]",NaN,20210-005,NaN,NaN,NaN


# Define information of interest and filter dataframe #

In [159]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,address,postalCode,crossStreet,neighborhood,id
0,Viaduto dos Fuzileiros,Bridge,-22.910323,-43.209499,"[{'label': 'display', 'lat': -22.9103228674344...",77,BR,Rio de Janeiro,RJ,Brasil,"[Rio de Janeiro, RJ, Brasil]",NaN,NaN,NaN,NaN,50c707f7e4b03c61133b0573
1,Viaduto dos Marinheiros,Bridge,-22.910619,-43.213063,"[{'label': 'display', 'lat': -22.9106185055282...",380,BR,NaN,NaN,Brasil,[Brasil],NaN,NaN,NaN,NaN,50ca435ee4b047cd3cd92e78
2,Botafogo Mercado De Flores,Flower Shop,-22.910233,-43.209925,"[{'label': 'display', 'lat': -22.9102330572094...",103,BR,Rio de Janeiro,RJ,Brasil,"[Rio de Janeiro, RJ, Brasil]",NaN,NaN,NaN,NaN,5128e368e4b02e93a4821aea
3,Centro de Convenções SulAmérica,Convention Center,-22.912070,-43.208819,"[{'label': 'display', 'lat': -22.9120699311280...",130,BR,Rio de Janeiro,RJ,Brasil,"[Av. Paulo de Frontin, 1, Rio de Janeiro, RJ, ...","Av. Paulo de Frontin, 1",20260-010,NaN,NaN,4d9a43728163a35d8077577e
4,Expo Cake Brasil,Fair,-22.912203,-43.209106,"[{'label': 'display', 'lat': -22.912203, 'lng'...",135,BR,Rio de Janeiro,RJ,Brasil,"[Rio de Janeiro, RJ, 20210-005, Brasil]",NaN,20210-005,NaN,NaN,5bc8c85b2b9844002cf18446


In [160]:
dataframe_filtered.name

0                                Viaduto dos Fuzileiros
1                               Viaduto dos Marinheiros
2                            Botafogo Mercado De Flores
3                       Centro de Convenções SulAmérica
4                                      Expo Cake Brasil
5                                        Gastro Service
6             Auditorio Principal - Centro De Conveções
7                                      Edted E-commerce
8                                   Confeitaria Querino
9                                           La Fattoria
10                            Pracinha do Tomé de Souza
11                           Edifício Martins Correa Sá
12                                   SulAmérica Seguros
13                                     Espaco Bem Estar
14                      SulAmérica Seguros - 3 andar TI
15                                 INTECH Engenharia C8
16                                 Universas Sulamerica
17                       Estacionamento Rio Cida

In [161]:
dataframe_filtered.categories

0                                       Bridge
1                                       Bridge
2                                  Flower Shop
3                            Convention Center
4                                         Fair
5                              Doctor's Office
6                                         None
7                            Convention Center
8                                       Bakery
9                                         Café
10                              Scenic Lookout
11    Residential Building (Apartment / Condo)
12                                      Office
13                           Convention Center
14                                Tech Startup
15                                        Fair
16                          College Auditorium
17                                     Parking
18                                        Fair
19                                   Cafeteria
20                                      Office
21           

In [162]:
dataframe_filtered.columns

Index(['name', 'categories', 'lat', 'lng', 'labeledLatLngs', 'distance', 'cc',
       'city', 'state', 'country', 'formattedAddress', 'address', 'postalCode',
       'crossStreet', 'neighborhood', 'id'],
      dtype='object')

In [144]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='School',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
       # popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

# Let's visualize these items on the map around our location #

In [131]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
       # popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map